In [1]:
%pip install duckdb sqlalchemy




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 26.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ArgumentError: Could not parse SQLAlchemy URL from string '/workspaces/ai_researcher_sqlebdesk/._dataset_2.db'

In [2]:
import duckdb
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker


In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import duckdb
import sqlite3

In [8]:


# Step 1: Connect to DuckDB and read data
duckdb_connection = duckdb.connect('_dataset_2.db')  # Connect to DuckDB

# Get the list of tables in the DuckDB database
tables_result = duckdb_connection.execute("SHOW TABLES").fetchall()
tables = [table[0] for table in tables_result]
print("Tables in DuckDB:", tables)

# Step 2: Connect to SQLite and create a new database
sqlite_connection = sqlite3.connect('sqlite_migrated.db')  # This will create a new SQLite DB
sqlite_cursor = sqlite_connection.cursor()

# Step 3: Transfer each table from DuckDB to SQLite
for table in tables:
    # Fetch the table schema from DuckDB
    table_schema = duckdb_connection.execute(f"PRAGMA table_info({table})").fetchall()
    
    # Create the same table in SQLite
    columns = ", ".join([f"{col[1]} {col[2]}" for col in table_schema])
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table} ({columns})"
    sqlite_cursor.execute(create_table_sql)

    # Fetch all data from the DuckDB table
    data = duckdb_connection.execute(f"SELECT * FROM {table}").fetchall()

    # Insert the data into the corresponding SQLite table
    placeholders = ", ".join(["?" for _ in range(len(table_schema))])
    insert_sql = f"INSERT INTO {table} VALUES ({placeholders})"
    
    sqlite_cursor.executemany(insert_sql, data)

# Step 4: Commit and close the connections
sqlite_connection.commit()
sqlite_connection.close()
duckdb_connection.close()

print("Data successfully migrated from DuckDB to SQLite!")


Tables in DuckDB: []
Data successfully migrated from DuckDB to SQLite!


In [ ]:
import duckdb

# Step 1: Connect to DuckDB and read data
duckdb_connection = duckdb.connect('_dataset_2.db')  # Connect to DuckDB

# Get the list of tables in the DuckDB database
tables_result = duckdb_connection.execute("SHOW TABLES").fetchall()
tables = [table[0] for table in tables_result]

# Show all available tables
print("Tables in DuckDB:")
for idx, table_name in enumerate(tables, 1):
    print(f"{idx}. {table_name}")

# Step 2: Ask the user to select a table by its index
table_index = int(input("\nEnter the number of the table you want to view: ")) - 1

if 0 <= table_index < len(tables):
    selected_table = tables[table_index]
    print(f"\nYou selected table: {selected_table}")
    
    # Fetch column information for the selected table
    columns_info = duckdb_connection.execute(f"PRAGMA table_info({selected_table})").fetchall()
    columns = [col[1] for col in columns_info]
    
    # Show available columns
    print(f"\nColumns in {selected_table}:")
    for idx, col_name in enumerate(columns, 1):
        print(f"{idx}. {col_name}")

    # Step 3: Ask the user to select a column for aggregation
    column_index = int(input("\nEnter the number of the column you want to aggregate: ")) - 1
    
    if 0 <= column_index < len(columns):
        selected_column = columns[column_index]
        print(f"\nYou selected column: {selected_column}")
        
        # Step 4: Ask the user to select the type of aggregation
        print("\nAvailable aggregation functions:")
        print("1. SUM")
        print("2. COUNT")
        print("3. AVG")
        print("4. MIN")
        print("5. MAX")
        
        agg_choice = int(input("\nEnter the number of the aggregation function you want to perform: "))
        
        # Map the user's choice to the correct SQL function
        agg_functions = {1: "SUM", 2: "COUNT", 3: "AVG", 4: "MIN", 5: "MAX"}
        if agg_choice in agg_functions:
            agg_function = agg_functions[agg_choice]
            
            # Step 5: Perform the aggregation query
            query = f"SELECT {agg_function}({selected_column}) FROM {selected_table}"
            result = duckdb_connection.execute(query).fetchone()
            
            print(f"\nResult of {agg_function}({selected_column}) on table {selected_table}: {result[0]}")
        else:
            print("Invalid aggregation choice.")
    
    else:
        print("Invalid column selection!")
    
else:
    print("Invalid table selection!")

# Close DuckDB connection
duckdb_connection.close()


Tables in DuckDB:
